# Getting Started with ModernBERT & GLUE

Created by: [Wayde Gilliam](https://twitter.com/waydegilliam)

## Encoders Strike Back!

Like many, I have fond memories of finetuning deberta, roberta and bert models for a number of Kaggle comps and real-world problems (e.g., NER, sentiment analysis, etc.).  Encoder models were "the thing" back in the day and continue to be the primary workhorse for many ML pipelines today though they have been eclipsed by recent advancements in LLMs which typically are based on decoder-only architectures. Long have we awaited a return to an encoder model for the modern world. With ModernBERT, that wait is over! ModernBERT is a new encoder-only model that incorporates the latest features in making neural networks more efficient, faster, and better at handling tasks that encoder models have long excelled at such at text classification.  In addition, ModernBERT allows us to break out of that max 512 token limit with their long context capabilities which give us 8,192 tokens to play with.

In this tutorial, we'll go through the steps of fine-tuning ModernBERT for one of the GLUE tasks, MRPC.  We'll cover some key settings required to use it with the HuggingFace trainer and include with some recommended hyperparameters that have served us well in fine-tuning ModernBERT for GLUE.  We'll also see how to use the model for inference and cleanup the model from the GPU to free up resources.

As an aside, I'm running all this code on a single 3090 with plenty of GPU memory to spare.

Though not strictly necessary, **ModernBERT trains better with FlashAttention!**. Training and inference will be much faster with it installed. See below:

ModernBERT is built on top of FlashAttention which is a highly optimized implementation of the attention mechanism that is faster and more memory efficient than the standard implementation.  ***The beauty of this is all you need to do is install it for ModernBERT to work with it!***  Here's how ...

For NVIDIA GPUs with compute capability 8.0+ (Ampere/Ada/Hopper architecture - A100, A6000, RTX 3090, RTX 4090, H100 etc):
```python
pip install flash-attn --no-build-isolation
```

For older NVIDIA GPUs (pre-Ampere):
```python
pip install flash-attn --no-deps
```


In [ ]:
#! pip install setuptools transformers datasets accelerate scikit-learn -Uqq
# install setuptools and do this before installing flash-attn
# pip install flash-attn --no-build-isolation


In [2]:
import os

os.environ["CUDA_VISIBLE_DEVICES"] = "0"


In [3]:
import numpy as np
import pandas as pd
import torch
from functools import partial
import gc

from datasets import load_dataset
from sklearn.metrics import f1_score
from transformers import (
    AutoTokenizer,
    AutoModelForSequenceClassification,
    DataCollatorWithPadding,
    TrainingArguments,
    Trainer,
    TrainerCallback,
)

from sklearn.metrics import matthews_corrcoef, accuracy_score, f1_score
from scipy.stats import pearsonr, spearmanr

os.environ["TOKENIZERS_PARALLELISM"] = "false"


/home/ubuntu/development/ModernBERT/.venv/lib/python3.11/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


## What is GLUE?

The [General Language Understanding Evaluation (GLUE) benchmark](https://gluebenchmark.com/) is a collection of nine diverse natural language understanding tasks designed to evaluate and compare the performance of NLP models across various language comprehension challenges. By providing a standardized framework, GLUE facilitates the development of models that generalize well across multiple tasks, promoting advancements in creating robust and versatile language understanding systems. 

Let's put this all these tasks in a dictionary along with some other helpful metadata about each one that might prove useful to iteratting over all of them.



In [29]:
# Diffusion 1.5B
glue_tasks = {
    "cola": {
        "abbr": "CoLA",
        "name": "Corpus of Linguistic Acceptability",
        "description": "Predict whether a sequence is a grammatical English sentence",
        "task_type": "Single-Sentence Task",
        "domain": "Misc.",
        "size": "8.5k",
        "metrics": "Matthews corr.",
        "dataset_names": {"train": "train", "valid": "validation", "test": "test"},
        "inputs": ["sentence"],
        "target": "label",
        "metric_funcs": [matthews_corrcoef, accuracy_score],
        "n_labels": 2,
        "learning_rate": 5e-5,
        "weight_decay": 8e-6,
        "epochs": 2
    },
    "sst2": {
        "abbr": "SST-2",
        "name": "Stanford Sentiment Treebank",
        "description": "Predict the sentiment of a given sentence",
        "task_type": "Single-Sentence Task",
        "domain": "Movie reviews",
        "size": "67k",
        "metrics": "Accuracy",
        "dataset_names": {"train": "train", "valid": "validation", "test": "test"},
        "inputs": ["sentence"],
        "target": "label",
        "metric_funcs": [accuracy_score],
        "n_labels": 2,
        "learning_rate": 8e-5,
        "weight_decay": 1e-5,
        "epochs": 2
    },
    "mrpc": {
        "abbr": "MRPC",
        "name": "Microsoft Research Paraphrase Corpus",
        "description": "Predict whether two sentences are semantically equivalent",
        "task_type": "Similarity and Paraphrase Tasks",
        "domain": "News",
        "size": "3.7k",
        "metrics": "F1/Accuracy",
        "dataset_names": {"train": "train", "valid": "validation", "test": "test"},
        "inputs": ["sentence1", "sentence2"],
        "target": "label",
        "metric_funcs": [accuracy_score, f1_score],
        "n_labels": 2,
        "learning_rate": 8e-5,
        "weight_decay": 1e-6,
        "epochs": 2
    },
    "stsb": {
        "abbr": "SST-B",
        "name": "Semantic Textual Similarity Benchmark",
        "description": "Predict the similarity score for two sentences on a scale from 1 to 5",
        "task_type": "Similarity and Paraphrase Tasks",
        "domain": "Misc.",
        "size": "7k",
        "metrics": "Pearson/Spearman corr.",
        "dataset_names": {"train": "train", "valid": "validation", "test": "test"},
        "inputs": ["sentence1", "sentence2"],
        "target": "label",
        "metric_funcs": [pearsonr, spearmanr, accuracy_score],
        "n_labels": 1,
        "learning_rate": 8e-5,
        "weight_decay": 1e-6,
        "epochs": 7
    },
    "qqp": {
        "abbr": "QQP",
        "name": "Quora question pair",
        "description": "Predict if two questions are a paraphrase of one another",
        "task_type": "Similarity and Paraphrase Tasks",
        "domain": "Social QA questions",
        "size": "364k",
        "metrics": "F1/Accuracy",
        "dataset_names": {"train": "train", "valid": "validation", "test": "test"},
        "inputs": ["question1", "question2"],
        "target": "label",
        "metric_funcs": [f1_score, accuracy_score],
        "n_labels": 2,
        "learning_rate": 8e-5,
        "weight_decay": 1e-5,
        "epochs": 10
    },
    "mnli-matched": {
        "abbr": "MNLI",
        "name": "Mulit-Genre Natural Language Inference",
        "description": "Predict whether the premise entails, contradicts or is neutral to the hypothesis",
        "task_type": "Inference Tasks",
        "domain": "Misc.",
        "size": "393k",
        "metrics": "Accuracy",
        "dataset_names": {"train": "train", "valid": "validation_matched", "test": "test_matched"},
        "inputs": ["premise", "hypothesis"],
        "target": "label",
        "metric_funcs": [accuracy_score],
        "n_labels": 3,
        "learning_rate": 8e-5,
        "weight_decay": 8e-6,
        "epochs": 2
    },
    "mnli-mismatched": {
        "abbr": "MNLI",
        "name": "Mulit-Genre Natural Language Inference",
        "description": "Predict whether the premise entails, contradicts or is neutral to the hypothesis",
        "task_type": "Inference Tasks",
        "domain": "Misc.",
        "size": "393k",
        "metrics": "Accuracy",
        "dataset_names": {"train": "train", "valid": "validation_mismatched", "test": "test_mismatched"},
        "inputs": ["premise", "hypothesis"],
        "target": "label",
        "metric_funcs": [accuracy_score],
        "n_labels": 3,
        "learning_rate": 8e-5,
        "weight_decay": 1e-5,
        "epochs": 2
    },
    "qnli": {
        "abbr": "QNLI",
        "name": "Stanford Question Answering Dataset",
        "description": "Predict whether the context sentence contains the answer to the question",
        "task_type": "Inference Tasks",
        "domain": "Wikipedia",
        "size": "105k",
        "metrics": "Accuracy",
        "dataset_names": {"train": "train", "valid": "validation", "test": "test"},
        "inputs": ["question", "sentence"],
        "target": "label",
        "metric_funcs": [accuracy_score],
        "n_labels": 2,
        "learning_rate": 8e-5,
        "weight_decay": 1e-5,
        "epochs": 3
    },
    "rte": {
        "abbr": "RTE",
        "name": "Recognize Textual Entailment",
        "description": "Predict whether one sentece entails another",
        "task_type": "Inference Tasks",
        "domain": "News, Wikipedia",
        "size": "2.5k",
        "metrics": "Accuracy",
        "dataset_names": {"train": "train", "valid": "validation", "test": "test"},
        "inputs": ["sentence1", "sentence2"],
        "target": "label",
        "metric_funcs": [accuracy_score],
        "n_labels": 2,
        "learning_rate": 8e-5,
        "weight_decay": 1e-6,
        "epochs": 10
    },
}

# for v in glue_tasks.values(): print(v)
glue_tasks.values()

glue_df = pd.DataFrame(glue_tasks.values(), columns=["abbr", "name", "task_type", "description", "size", "metrics"])
glue_df.columns = glue_df.columns.str.replace("_", " ").str.capitalize()
display(glue_df.style.set_properties(**{"text-align": "left"}))

,Abbr,Name,Task type,Description,Size,Metrics
0,CoLA,Corpus of Linguistic Acceptability,Single-Sentence Task,Predict whether a sequence is a grammatical English sentence,8.5k,Matthews corr.
1,SST-2,Stanford Sentiment Treebank,Single-Sentence Task,Predict the sentiment of a given sentence,67k,Accuracy
2,MRPC,Microsoft Research Paraphrase Corpus,Similarity and Paraphrase Tasks,Predict whether two sentences are semantically equivalent,3.7k,F1/Accuracy
3,SST-B,Semantic Textual Similarity Benchmark,Similarity and Paraphrase Tasks,Predict the similarity score for two sentences on a scale from 1 to 5,7k,Pearson/Spearman corr.
4,QQP,Quora question pair,Similarity and Paraphrase Tasks,Predict if two questions are a paraphrase of one another,364k,F1/Accuracy
5,MNLI,Mulit-Genre Natural Language Inference,Inference Tasks,"Predict whether the premise entails, contradicts or is neutral to the hypothesis",393k,Accuracy
6,MNLI,Mulit-Genre Natural Language Inference,Inference Tasks,"Predict whether the premise entails, contradicts or is neutral to the hypothesis",393k,Accuracy
7,QNLI,Stanford Question Answering Dataset,Inference Tasks,Predict whether the context sentence contains the answer to the question,105k,Accuracy
8,RTE,Recognize Textual Entailment,Inference Tasks,Predict whether one sentece entails another,2.5k,Accuracy


In [ ]:
# Mask 1.5B
glue_tasks = {
    "cola": {
        "abbr": "CoLA",
        "name": "Corpus of Linguistic Acceptability",
        "description": "Predict whether a sequence is a grammatical English sentence",
        "task_type": "Single-Sentence Task",
        "domain": "Misc.",
        "size": "8.5k",
        "metrics": "Matthews corr.",
        "dataset_names": {"train": "train", "valid": "validation", "test": "test"},
        "inputs": ["sentence"],
        "target": "label",
        "metric_funcs": [matthews_corrcoef, accuracy_score],
        "n_labels": 2,
        "learning_rate": 1e-5,
        "weight_decay": 1e-6,
        "epochs": 9
    },
    "sst2": {
        "abbr": "SST-2",
        "name": "Stanford Sentiment Treebank",
        "description": "Predict the sentiment of a given sentence",
        "task_type": "Single-Sentence Task",
        "domain": "Movie reviews",
        "size": "67k",
        "metrics": "Accuracy",
        "dataset_names": {"train": "train", "valid": "validation", "test": "test"},
        "inputs": ["sentence"],
        "target": "label",
        "metric_funcs": [accuracy_score],
        "n_labels": 2,
        "learning_rate": 5e-5,
        "weight_decay": 1e-5,
        "epochs": 6
    },
    "mrpc": {
        "abbr": "MRPC",
        "name": "Microsoft Research Paraphrase Corpus",
        "description": "Predict whether two sentences are semantically equivalent",
        "task_type": "Similarity and Paraphrase Tasks",
        "domain": "News",
        "size": "3.7k",
        "metrics": "F1/Accuracy",
        "dataset_names": {"train": "train", "valid": "validation", "test": "test"},
        "inputs": ["sentence1", "sentence2"],
        "target": "label",
        "metric_funcs": [accuracy_score, f1_score],
        "n_labels": 2,
        "learning_rate": 8e-5,
        "weight_decay": 1e-6,
        "epochs": 2
    },
    "stsb": {
        "abbr": "SST-B",
        "name": "Semantic Textual Similarity Benchmark",
        "description": "Predict the similarity score for two sentences on a scale from 1 to 5",
        "task_type": "Similarity and Paraphrase Tasks",
        "domain": "Misc.",
        "size": "7k",
        "metrics": "Pearson/Spearman corr.",
        "dataset_names": {"train": "train", "valid": "validation", "test": "test"},
        "inputs": ["sentence1", "sentence2"],
        "target": "label",
        "metric_funcs": [pearsonr, spearmanr, accuracy_score],
        "n_labels": 1,
        "learning_rate": 8e-5,
        "weight_decay": 8e-6,
        "epochs": 9
    },
    "qqp": {
        "abbr": "QQP",
        "name": "Quora question pair",
        "description": "Predict if two questions are a paraphrase of one another",
        "task_type": "Similarity and Paraphrase Tasks",
        "domain": "Social QA questions",
        "size": "364k",
        "metrics": "F1/Accuracy",
        "dataset_names": {"train": "train", "valid": "validation", "test": "test"},
        "inputs": ["question1", "question2"],
        "target": "label",
        "metric_funcs": [f1_score, accuracy_score],
        "n_labels": 2,
        "learning_rate": 8e-5,
        "weight_decay": 1e-5,
        "epochs": 10
    },
    "mnli-matched": {
        "abbr": "MNLI",
        "name": "Mulit-Genre Natural Language Inference",
        "description": "Predict whether the premise entails, contradicts or is neutral to the hypothesis",
        "task_type": "Inference Tasks",
        "domain": "Misc.",
        "size": "393k",
        "metrics": "Accuracy",
        "dataset_names": {"train": "train", "valid": "validation_matched", "test": "test_matched"},
        "inputs": ["premise", "hypothesis"],
        "target": "label",
        "metric_funcs": [accuracy_score],
        "n_labels": 3,
        "learning_rate": 8e-5,
        "weight_decay": 1e-6,
        "epochs": 5
    },
    "mnli-mismatched": {
        "abbr": "MNLI",
        "name": "Mulit-Genre Natural Language Inference",
        "description": "Predict whether the premise entails, contradicts or is neutral to the hypothesis",
        "task_type": "Inference Tasks",
        "domain": "Misc.",
        "size": "393k",
        "metrics": "Accuracy",
        "dataset_names": {"train": "train", "valid": "validation_mismatched", "test": "test_mismatched"},
        "inputs": ["premise", "hypothesis"],
        "target": "label",
        "metric_funcs": [accuracy_score],
        "n_labels": 3,
        "learning_rate": 8e-5,
        "weight_decay": 1e-6,
        "epochs": 4
    },
    "qnli": {
        "abbr": "QNLI",
        "name": "Stanford Question Answering Dataset",
        "description": "Predict whether the context sentence contains the answer to the question",
        "task_type": "Inference Tasks",
        "domain": "Wikipedia",
        "size": "105k",
        "metrics": "Accuracy",
        "dataset_names": {"train": "train", "valid": "validation", "test": "test"},
        "inputs": ["question", "sentence"],
        "target": "label",
        "metric_funcs": [accuracy_score],
        "n_labels": 2,
        "learning_rate": 5e-5,
        "weight_decay": 5e-6,
        "epochs": 4
    },
    "rte": {
        "abbr": "RTE",
        "name": "Recognize Textual Entailment",
        "description": "Predict whether one sentece entails another",
        "task_type": "Inference Tasks",
        "domain": "News, Wikipedia",
        "size": "2.5k",
        "metrics": "Accuracy",
        "dataset_names": {"train": "train", "valid": "validation", "test": "test"},
        "inputs": ["sentence1", "sentence2"],
        "target": "label",
        "metric_funcs": [accuracy_score],
        "n_labels": 2,
        "learning_rate": 8e-5,
        "weight_decay": 1e-6,
        "epochs": 4
    },
}

# for v in glue_tasks.values(): print(v)
glue_tasks.values()

glue_df = pd.DataFrame(glue_tasks.values(), columns=["abbr", "name", "task_type", "description", "size", "metrics"])
glue_df.columns = glue_df.columns.str.replace("_", " ").str.capitalize()
display(glue_df.style.set_properties(**{"text-align": "left"}))

,Abbr,Name,Task type,Description,Size,Metrics
0,CoLA,Corpus of Linguistic Acceptability,Single-Sentence Task,Predict whether a sequence is a grammatical English sentence,8.5k,Matthews corr.
1,SST-2,Stanford Sentiment Treebank,Single-Sentence Task,Predict the sentiment of a given sentence,67k,Accuracy
2,MRPC,Microsoft Research Paraphrase Corpus,Similarity and Paraphrase Tasks,Predict whether two sentences are semantically equivalent,3.7k,F1/Accuracy
3,SST-B,Semantic Textual Similarity Benchmark,Similarity and Paraphrase Tasks,Predict the similarity score for two sentences on a scale from 1 to 5,7k,Pearson/Spearman corr.
4,QQP,Quora question pair,Similarity and Paraphrase Tasks,Predict if two questions are a paraphrase of one another,364k,F1/Accuracy
5,MNLI,Mulit-Genre Natural Language Inference,Inference Tasks,"Predict whether the premise entails, contradicts or is neutral to the hypothesis",393k,Accuracy
6,MNLI,Mulit-Genre Natural Language Inference,Inference Tasks,"Predict whether the premise entails, contradicts or is neutral to the hypothesis",393k,Accuracy
7,QNLI,Stanford Question Answering Dataset,Inference Tasks,Predict whether the context sentence contains the answer to the question,105k,Accuracy
8,RTE,Recognize Textual Entailment,Inference Tasks,Predict whether one sentece entails another,2.5k,Accuracy


## Let's Fine-Tune ModernBERT for MRPC

### Configuration

ModernBERT currently comes in two flavors, base and large. To keep things lean and mean, we'll use the "answerdotai/ModernBERT-base" checkpoint for this example.

In [5]:
task = "mrpc"
task_meta = glue_tasks[task]
train_ds_name = task_meta["dataset_names"]["train"]
valid_ds_name = task_meta["dataset_names"]["valid"]
test_ds_name = task_meta["dataset_names"]["test"]

task_inputs = task_meta["inputs"]
task_target = task_meta["target"]
n_labels = task_meta["n_labels"]
task_metrics = task_meta["metric_funcs"]

checkpoint = "output_model/modernbert-diffusion-1b"  # "answerdotai/ModernBERT-base", "answerdotai/ModernBERT-large"

KeyError: 'mrpc'

### Data

We'll use the `Datasets` library to load the data.  As its always recommended to "look at your data" before we get training, we'll also print out a single example to see what we're working with as well as the features of the dataset.

In [5]:
raw_datasets = load_dataset("glue", task)

print(f"{raw_datasets}\n")
print(f"{raw_datasets[train_ds_name][0]}\n")
print(f"{raw_datasets[train_ds_name].features}\n")

DatasetDict({
    train: Dataset({
        features: ['sentence1', 'sentence2', 'label', 'idx'],
        num_rows: 3668
    })
    validation: Dataset({
        features: ['sentence1', 'sentence2', 'label', 'idx'],
        num_rows: 408
    })
    test: Dataset({
        features: ['sentence1', 'sentence2', 'label', 'idx'],
        num_rows: 1725
    })
})

{'sentence1': 'Amrozi accused his brother , whom he called " the witness " , of deliberately distorting his evidence .', 'sentence2': 'Referring to him as only " the witness " , Amrozi accused his brother of deliberately distorting his evidence .', 'label': 1, 'idx': 0}

{'sentence1': Value('string'), 'sentence2': Value('string'), 'label': ClassLabel(names=['not_equivalent', 'equivalent']), 'idx': Value('int32')}



We can use the following dictionaries when building our model with `AutoModelForSequenceClassification` to map between the label ids and names.

In [5]:
def get_label_maps(raw_datasets, train_ds_name):
    labels = raw_datasets[train_ds_name].features["label"]

    id2label = {idx: name.upper() for idx, name in enumerate(labels.names)} if hasattr(labels, "names") else None
    label2id = {name.upper(): idx for idx, name in enumerate(labels.names)} if hasattr(labels, "names") else None

    return id2label, label2id

In [7]:
id2label, label2id = get_label_maps(raw_datasets, train_ds_name)

print(f"{id2label}")
print(f"{label2id}")


{0: 'NOT_EQUIVALENT', 1: 'EQUIVALENT'}
{'NOT_EQUIVALENT': 0, 'EQUIVALENT': 1}


MRPC is a sentence-pair classification task where we're given two sentences and asked to predict whether they are paraphrases of one another.  The dataset is split into train, validation and test sets. We'll need to keep all this in mind when we set up tokenization next with `AutoTokenizer`.

### Tokenizer

Next we define our Tokenizer and a preprocess function to create the input_ids, attention_mask, and token_type_ids the model nees to train.  For this example, including `truncation=True` is enough as we'll rely on our data collation function below to put our batches into the correct shape.

In [8]:
hf_tokenizer = AutoTokenizer.from_pretrained(checkpoint)

In [9]:
task_inputs

['sentence1', 'sentence2']

In [6]:
def preprocess_function(examples, task_inputs):
    inps = [examples[inp] for inp in task_inputs]
    tokenized = hf_tokenizer(*inps, truncation=True)
    return tokenized

In [12]:
tokenized_datasets = raw_datasets.map(partial(preprocess_function, task_inputs=task_inputs), batched=True)

print(f"{tokenized_datasets}\n")
print(f"{tokenized_datasets[train_ds_name][0]}\n")
print(f"{tokenized_datasets[train_ds_name].features}\n")

DatasetDict({
    train: Dataset({
        features: ['sentence1', 'sentence2', 'label', 'idx', 'input_ids', 'attention_mask'],
        num_rows: 3668
    })
    validation: Dataset({
        features: ['sentence1', 'sentence2', 'label', 'idx', 'input_ids', 'attention_mask'],
        num_rows: 408
    })
    test: Dataset({
        features: ['sentence1', 'sentence2', 'label', 'idx', 'input_ids', 'attention_mask'],
        num_rows: 1725
    })
})

{'sentence1': 'Amrozi accused his brother , whom he called " the witness " , of deliberately distorting his evidence .', 'sentence2': 'Referring to him as only " the witness " , Amrozi accused his brother of deliberately distorting his evidence .', 'label': 1, 'idx': 0, 'input_ids': [50281, 8096, 287, 9877, 10145, 521, 4929, 1157, 5207, 344, 1925, 346, 253, 5517, 346, 1157, 273, 21547, 940, 12655, 521, 1941, 964, 50282, 7676, 24247, 281, 779, 347, 760, 346, 253, 5517, 346, 1157, 3052, 287, 9877, 10145, 521, 4929, 273, 21547, 940, 12655, 521,

It's always good to see what the tokenizer is doing to our data to ensure the special tokens are where we expect them to be!

In [13]:
hf_tokenizer.decode(tokenized_datasets[train_ds_name][0]["input_ids"])

'[CLS]Amrozi accused his brother, whom he called " the witness ", of deliberately distorting his evidence.[SEP]Referring to him as only " the witness ", Amrozi accused his brother of deliberately distorting his evidence.[SEP]'

### Metrics

We'll use our `task_metrics` to compute the metrics for our model.  We'll return a dictionary of the metric name and value for each metric we're interested in.

In [19]:
def compute_metrics(eval_pred, task_metrics):
    predictions, labels = eval_pred

    metrics_d = {}
    for metric_func in task_metrics:
        metric_name = metric_func.__name__
        if metric_name in ["pearsonr", "spearmanr"]:
            score = metric_func(labels, np.squeeze(predictions))
        elif metric_name == "accuracy_score":
            # For STSB: threshold-based binary accuracy
            if len(predictions.shape) == 1 or predictions.shape[1] == 1:
                # Regression task (STSB) - convert to binary classification
                pred_binary = (np.squeeze(predictions) >= 3.0).astype(int)
                label_binary = (labels >= 3.0).astype(int)
                score = metric_func(label_binary, pred_binary)
            else:
                # Classification task - use argmax
                score = metric_func(np.argmax(predictions, axis=-1), labels)
        else:
            # Other classification metrics
            if len(predictions.shape) > 1 and predictions.shape[1] > 1:
                score = metric_func(np.argmax(predictions, axis=-1), labels)
            else:
                # For regression, most metrics don't apply
                continue

        if isinstance(score, tuple):
            metrics_d[metric_func.__name__] = score[0]
        else:
            metrics_d[metric_func.__name__] = score

    return metrics_d

### Train

This is where the fun begins! Here we setup a few hyperparameters than have proven to work well for us in fine-tuning ModernBERT-base on GLUE tasks.  We'll also setup our model, data collator, and training arguments.

In [15]:
train_bsz, val_bsz = 32, 32
lr = 8e-5
betas = (0.9, 0.98)
n_epochs = 2
eps = 1e-6
wd = 8e-6

When configuring `AutoModelForSequenceClassification`, two settings are critical to get things working with the HuggingFace `Trainer`. One is the `num_labels` we're expecting and the other is to set `compile=False` to avoid using the `torch.compile` function which is not supported in Transformers at the time of this writing.

In [16]:
hf_model = AutoModelForSequenceClassification.from_pretrained(
    checkpoint, num_labels=n_labels, id2label=id2label, label2id=label2id
)


Flash Attention 2 only supports torch.float16 and torch.bfloat16 dtypes, but the current dype in ModernBertForSequenceClassification is torch.float32. You should run training or inference using Automatic Mixed-Precision via the `with torch.autocast(device_type='torch_device'):` decorator, or load the model with the `dtype` argument. Example: `model = AutoModel.from_pretrained("openai/whisper-tiny", attn_implementation="flash_attention_2", dtype=torch.float16)`
Flash Attention 2 only supports torch.float16 and torch.bfloat16 dtypes, but the current dype in ModernBertModel is torch.float32. You should run training or inference using Automatic Mixed-Precision via the `with torch.autocast(device_type='torch_device'):` decorator, or load the model with the `dtype` argument. Example: `model = AutoModel.from_pretrained("openai/whisper-tiny", attn_implementation="flash_attention_2", dtype=torch.float16)`
Some weights of ModernBertForSequenceClassification were not initialized from the model ch

Collation is easy for GLUE tasks as we can use the `DataCollatorWithPadding` class to pad our input_ids and attention_mask to the max length in the batch.

**Note**: If you have installed Flash Attention, ModernBERT removes the padding internally, which makes it the fastest version. SPDA and Eager mode will be slower.

In [17]:
hf_data_collator = DataCollatorWithPadding(tokenizer=hf_tokenizer)

With all the pieces in place, we can now setup our `TrainingArguments` and `Trainer` and get to training! Lots of customization is possible here and it is recommended to play with different schedulers and the hyperparameters we've started y'all off with above to improve results.

In [18]:
training_args = TrainingArguments(
    output_dir=f"aai_ModernBERT_{task}_ft",
    learning_rate=lr,
    per_device_train_batch_size=train_bsz,
    per_device_eval_batch_size=val_bsz,
    num_train_epochs=n_epochs,
    lr_scheduler_type="linear",
    optim="adamw_torch",
    adam_beta1=betas[0],
    adam_beta2=betas[1],
    adam_epsilon=eps,
    logging_strategy="epoch",
    eval_strategy="epoch",
    save_strategy="epoch",
    load_best_model_at_end=True,
    bf16=True,
    bf16_full_eval=True,
    push_to_hub=False,
)

We define `TrainerCallback` so that we can capture all the training and evaluation logs and store them for later analysis. By default, the `Trainer` class will only keep the latest logs.


In [8]:
class MetricsCallback(TrainerCallback):
    def __init__(self):
        self.training_history = {"train": [], "eval": []}

    def on_log(self, args, state, control, logs=None, **kwargs):
        if logs is not None:
            if "loss" in logs:  # Training logs
                self.training_history["train"].append(logs)
            elif "eval_loss" in logs:  # Evaluation logs
                self.training_history["eval"].append(logs)

In [20]:
trainer = Trainer(
    model=hf_model,
    args=training_args,
    train_dataset=tokenized_datasets[train_ds_name],
    eval_dataset=tokenized_datasets[valid_ds_name],
    processing_class=hf_tokenizer,
    data_collator=hf_data_collator,
    compute_metrics=partial(compute_metrics, task_metrics=task_metrics),
)

metrics_callback = MetricsCallback()
trainer.add_callback(metrics_callback)

trainer.train()

train_history_df = pd.DataFrame(metrics_callback.training_history["train"])
train_history_df = train_history_df.add_prefix("train_")
eval_history_df = pd.DataFrame(metrics_callback.training_history["eval"])
train_res_df = pd.concat([train_history_df, eval_history_df], axis=1)

args_df = pd.DataFrame([training_args.to_dict()])

display(train_res_df)
display(args_df)

The tokenizer has new PAD/BOS/EOS tokens that differ from the model config and generation config. The model config and generation config were aligned accordingly, being updated with the tokenizer's values. Updated tokens: {'eos_token_id': None, 'bos_token_id': None}.


Epoch,Training Loss,Validation Loss,Accuracy Score,F1 Score
1,0.700400,0.612666,0.705882,0.815385
2,0.513900,0.601638,0.705882,0.807692


,train_loss,train_grad_norm,train_learning_rate,train_epoch,eval_loss,eval_accuracy_score,eval_f1_score
0,0.7004,3.449601,4.034783e-05,1.0,0.612666,0.705882,0.815385
1,0.5139,3.151908,3.478261e-07,2.0,0.601638,0.705882,0.807692


,output_dir,overwrite_output_dir,do_train,do_eval,do_predict,eval_strategy,prediction_loss_only,per_device_train_batch_size,per_device_eval_batch_size,per_gpu_train_batch_size,...,include_tokens_per_second,include_num_input_tokens_seen,neftune_noise_alpha,optim_target_modules,batch_eval_metrics,eval_on_start,use_liger_kernel,liger_kernel_config,eval_use_gather_object,average_tokens_across_devices
0,aai_ModernBERT_mrpc_ft,False,False,True,False,epoch,False,32,32,None,...,False,False,None,None,False,False,False,None,False,False


### Inference

There'a number of options for inference within the HuggingFace ecosystem.  We'll go a bit old school here and just use the `forward` method of the model. We're not uploading this model to the hub, but this is an easy enough task for you to try out on your own should you like to share your ModernBERT finetune :).

In [21]:
ex_1 = "The quick brown fox jumps over the lazy dog."
ex_2 = "I love lamp!"

inf_inputs = hf_tokenizer(ex_1, ex_2, return_tensors="pt")
inf_inputs = inf_inputs.to("cuda")

with torch.no_grad():
    logits = hf_model(**inf_inputs).logits

print(logits)
print(f"Prediction: {hf_model.config.id2label[logits.argmax().item()]}")


tensor([[2.8281, 2.3281]], device='cuda:0')
Prediction: NOT_EQUIVALENT


### Cleanup

In [9]:
def cleanup(things_to_delete: list | None = None):
    if things_to_delete is not None:
        for thing in things_to_delete:
            if thing is not None:
                del thing

    gc.collect()
    torch.cuda.empty_cache()


In [9]:
cleanup(things_to_delete=[hf_model, trainer])

NameError: name 'hf_model' is not defined

## Train all the GLUE!

If you got this far you're probably wondering why I put together that dictionary of GLUE tasks if all we're doing is finetuning a single model. The answer is basically that I'm a good and lazy programmer who would like to easily run hyperparameter sweeps and/or fine-tunes on all the GLUE tasks. So ... let's do that!

We'll run with the training hyperparameters specified above and I leave it to the reader to improve the method below to be able to override these values should folks be looking for something to do :)

In [13]:


def finetune_glue_task(
    task: str, checkpoint: str = "answerdotai/ModernBERT-base", train_subset: int | None = None, do_cleanup: bool = True
):  # 1. Load the task metadata
    task_meta = glue_tasks[task]
    train_ds_name = task_meta["dataset_names"]["train"]
    valid_ds_name = task_meta["dataset_names"]["valid"]

    task_inputs = task_meta["inputs"]
    n_labels = task_meta["n_labels"]
    task_metrics = task_meta["metric_funcs"]

    # 2. Load the dataset
    raw_datasets = load_dataset("glue", task.split("-")[0] if "-" in task else task)
    if train_subset is not None and len(raw_datasets["train"]) > train_subset:
        raw_datasets["train"] = raw_datasets["train"].shuffle(seed=42).select(range(train_subset))

    id2label, label2id = get_label_maps(raw_datasets, train_ds_name)

    # 3. Load the tokenizer
    hf_tokenizer = AutoTokenizer.from_pretrained("answerdotai/ModernBERT-base")
    tokenized_datasets = raw_datasets.map(partial(preprocess_function, task_inputs=task_inputs), batched=True)

    # 4. Define the compute metrics function
    task_compute_metrics = partial(compute_metrics, task_metrics=task_metrics)

    # 5. Load the model and data collator
    model_additional_kwargs = {"id2label": id2label, "label2id": label2id} if id2label and label2id else {}
    hf_model = AutoModelForSequenceClassification.from_pretrained(
        checkpoint, num_labels=n_labels, **model_additional_kwargs
    )

    hf_data_collator = DataCollatorWithPadding(tokenizer=hf_tokenizer)

    # 6. Define the training arguments and trainer
    training_args = TrainingArguments(
        output_dir=f"aai_ModernBERT_{task}_ft",
        learning_rate=lr,
        per_device_train_batch_size=train_bsz,
        per_device_eval_batch_size=val_bsz,
        num_train_epochs=n_epochs,
        lr_scheduler_type="linear",
        optim="adamw_torch",
        adam_beta1=betas[0],
        adam_beta2=betas[1],
        adam_epsilon=eps,
        logging_strategy="epoch",
        eval_strategy="epoch",
        save_strategy="epoch",
        load_best_model_at_end=True,
        bf16=True,
        bf16_full_eval=True,
        push_to_hub=False,
    )

    trainer = Trainer(
        model=hf_model,
        args=training_args,
        train_dataset=tokenized_datasets[train_ds_name],
        eval_dataset=tokenized_datasets[valid_ds_name],
        processing_class=hf_tokenizer,
        data_collator=hf_data_collator,
        compute_metrics=task_compute_metrics,
    )

    # Add callback to trainer
    metrics_callback = MetricsCallback()
    trainer.add_callback(metrics_callback)

    trainer.train()

    # 7. Get the training results and hyperparameters
    train_history_df = pd.DataFrame(metrics_callback.training_history["train"])
    train_history_df = train_history_df.add_prefix("train_")
    eval_history_df = pd.DataFrame(metrics_callback.training_history["eval"])
    train_res_df = pd.concat([train_history_df, eval_history_df], axis=1)

    args_df = pd.DataFrame([training_args.to_dict()])

    # 8. Cleanup (optional)
    if do_cleanup:
        cleanup(things_to_delete=[trainer, hf_model, hf_tokenizer, tokenized_datasets, raw_datasets])

    return train_res_df, args_df, hf_model, hf_tokenizer

This helpful function encapsulates all the steps we've been through above and allows us to easily run a fine-tune on a single task. In addition to the HuggingFace objects, it returns the training results, training hyperparameters (all potentially helpful for performing sweeps and or documenting your results).

Let's give it a go on both MRPC and CoLA.


In [24]:
train_res_df, args_df, hf_model, hf_tokenizer = finetune_glue_task(
    "mrpc", checkpoint=checkpoint, do_cleanup=True
)

display(train_res_df)
display(args_df)


Map: 100%|██████████| 1725/1725 [00:00<00:00, 14059.15 examples/s]
Some weights of ModernBertForSequenceClassification were not initialized from the model checkpoint at output_model/modernbert-diffusion-1b and are newly initialized: ['classifier.bias', 'classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.
The tokenizer has new PAD/BOS/EOS tokens that differ from the model config and generation config. The model config and generation config were aligned accordingly, being updated with the tokenizer's values. Updated tokens: {'eos_token_id': None, 'bos_token_id': None}.


Epoch,Training Loss,Validation Loss,Accuracy Score,F1 Score
1,0.724300,0.569719,0.730392,0.832827
2,0.508100,0.557317,0.740196,0.833856


,train_loss,train_grad_norm,train_learning_rate,train_epoch,eval_loss,eval_accuracy_score,eval_f1_score
0,0.7243,3.341632,4.034783e-05,1.0,0.569719,0.730392,0.832827
1,0.5081,4.132639,3.478261e-07,2.0,0.557317,0.740196,0.833856


,output_dir,overwrite_output_dir,do_train,do_eval,do_predict,eval_strategy,prediction_loss_only,per_device_train_batch_size,per_device_eval_batch_size,per_gpu_train_batch_size,...,include_tokens_per_second,include_num_input_tokens_seen,neftune_noise_alpha,optim_target_modules,batch_eval_metrics,eval_on_start,use_liger_kernel,liger_kernel_config,eval_use_gather_object,average_tokens_across_devices
0,aai_ModernBERT_mrpc_ft,False,False,True,False,epoch,False,32,32,None,...,False,False,None,None,False,False,False,None,False,False


In [25]:
train_res_df, args_df, hf_model, hf_tokenizer = finetune_glue_task(
    "cola", checkpoint=checkpoint, do_cleanup=True
)

display(train_res_df)
display(args_df)

Some weights of ModernBertForSequenceClassification were not initialized from the model checkpoint at output_model/modernbert-diffusion-1b and are newly initialized: ['classifier.bias', 'classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.
The tokenizer has new PAD/BOS/EOS tokens that differ from the model config and generation config. The model config and generation config were aligned accordingly, being updated with the tokenizer's values. Updated tokens: {'eos_token_id': None, 'bos_token_id': None}.


Epoch,Training Loss,Validation Loss,Matthews Corrcoef,Accuracy Score
1,0.640100,0.599142,0.098772,0.690316
2,0.510400,0.607198,0.123063,0.689358


,train_loss,train_grad_norm,train_learning_rate,train_epoch,eval_loss,eval_matthews_corrcoef,eval_accuracy_score
0,0.6401,3.880660,4.014925e-05,1.0,0.599142,0.098772,0.690316
1,0.5104,4.748318,1.492537e-07,2.0,0.607198,0.123063,0.689358


,output_dir,overwrite_output_dir,do_train,do_eval,do_predict,eval_strategy,prediction_loss_only,per_device_train_batch_size,per_device_eval_batch_size,per_gpu_train_batch_size,...,include_tokens_per_second,include_num_input_tokens_seen,neftune_noise_alpha,optim_target_modules,batch_eval_metrics,eval_on_start,use_liger_kernel,liger_kernel_config,eval_use_gather_object,average_tokens_across_devices
0,aai_ModernBERT_cola_ft,False,False,True,False,epoch,False,32,32,None,...,False,False,None,None,False,False,False,None,False,False


**Send it!**

Grab yourself a good cup of coffee, take your pups out for a walk, or whatever as your GPU purrs along while finetuning all things GLUE!

Note the `train_subset` parameter which allows us to train on a subset of the dataset. This is helpful for quickly testing the model on a small dataset to make sure all the bits work as expected.  Feel free to set it to `None` for a full send!

In [26]:
for task in glue_tasks.keys():
    print(f"----- Finetuning {task} -----")
    train_res_df, args_df, hf_model, hf_tokenizer = finetune_glue_task(
        task, checkpoint=checkpoint, train_subset=None, do_cleanup=True
    )

    print(":: Results ::")
    display(train_res_df)
    display(args_df)


----- Finetuning cola -----


Some weights of ModernBertForSequenceClassification were not initialized from the model checkpoint at output_model/modernbert-diffusion-1b and are newly initialized: ['classifier.bias', 'classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.
The tokenizer has new PAD/BOS/EOS tokens that differ from the model config and generation config. The model config and generation config were aligned accordingly, being updated with the tokenizer's values. Updated tokens: {'eos_token_id': None, 'bos_token_id': None}.


Epoch,Training Loss,Validation Loss,Matthews Corrcoef,Accuracy Score
1,0.640200,0.599160,0.109335,0.692234
2,0.510500,0.606568,0.120251,0.688399


:: Results ::


,train_loss,train_grad_norm,train_learning_rate,train_epoch,eval_loss,eval_matthews_corrcoef,eval_accuracy_score
0,0.6402,3.848574,4.014925e-05,1.0,0.599160,0.109335,0.692234
1,0.5105,4.709330,1.492537e-07,2.0,0.606568,0.120251,0.688399


...

Some weights of ModernBertForSequenceClassification were not initialized from the model checkpoint at output_model/modernbert-diffusion-1b and are newly initialized: ['classifier.bias', 'classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.
The tokenizer has new PAD/BOS/EOS tokens that differ from the model config and generation config. The model config and generation config were aligned accordingly, being updated with the tokenizer's values. Updated tokens: {'eos_token_id': None, 'bos_token_id': None}.


Epoch,Training Loss,Validation Loss,Accuracy Score
1,1.060400,0.911257,0.478873
2,0.785200,0.921793,0.394366


:: Results ::


,train_loss,train_grad_norm,train_learning_rate,train_epoch,eval_loss,eval_accuracy_score
0,1.0604,14.327180,0.000042,1.0,0.911257,0.478873
1,0.7852,6.899503,0.000002,2.0,0.921793,0.394366


,output_dir,overwrite_output_dir,do_train,do_eval,do_predict,eval_strategy,prediction_loss_only,per_device_train_batch_size,per_device_eval_batch_size,per_gpu_train_batch_size,...,include_tokens_per_second,include_num_input_tokens_seen,neftune_noise_alpha,optim_target_modules,batch_eval_metrics,eval_on_start,use_liger_kernel,liger_kernel_config,eval_use_gather_object,average_tokens_across_devices
0,aai_ModernBERT_wnli_ft,False,False,True,False,epoch,False,32,32,None,...,False,False,None,None,False,False,False,None,False,False


## Conclusion

With ModernBERT encoders are back baby!  We've seen that ModernBERT-base can compete with the best of them on GLUE tasks and with a little more tuning, we'll see that ModernBERT-large can do even better.  I'm excited to see what the community will do with this model and I'm looking forward to seeing what you all build with it! We'll be exploring more of the capabilities of ModernBERT in future tutorials.

Until next time, happy coding!


# Test All models

In [10]:
train_bsz, val_bsz = 32, 32
betas = (0.9, 0.98)
n_epochs = 10
eps = 1e-6

def finetune_glue_task(
    lr, wd, task: str, checkpoint: str = "answerdotai/ModernBERT-base", train_subset: int | None = None, do_cleanup: bool = True
):  # 1. Load the task metadata
    task_meta = glue_tasks[task]
    train_ds_name = task_meta["dataset_names"]["train"]
    valid_ds_name = task_meta["dataset_names"]["valid"]

    task_inputs = task_meta["inputs"]
    n_labels = task_meta["n_labels"]
    task_metrics = task_meta["metric_funcs"]

    # 2. Load the dataset
    raw_datasets = load_dataset("glue", task.split("-")[0] if "-" in task else task)
    if train_subset is not None and len(raw_datasets["train"]) > train_subset:
        raw_datasets["train"] = raw_datasets["train"].shuffle(seed=42).select(range(train_subset))

    id2label, label2id = get_label_maps(raw_datasets, train_ds_name)

    # 3. Load the tokenizer
    hf_tokenizer = AutoTokenizer.from_pretrained(checkpoint)
    tokenized_datasets = raw_datasets.map(partial(preprocess_function, task_inputs=task_inputs), batched=True)

    # 4. Define the compute metrics function
    task_compute_metrics = partial(compute_metrics, task_metrics=task_metrics)

    # 5. Load the model and data collator
    model_additional_kwargs = {"id2label": id2label, "label2id": label2id} if id2label and label2id else {}
    hf_model = AutoModelForSequenceClassification.from_pretrained(
        checkpoint, num_labels=n_labels, **model_additional_kwargs
    )

    hf_data_collator = DataCollatorWithPadding(tokenizer=hf_tokenizer)

    # 6. Define the training arguments and trainer
    training_args = TrainingArguments(
        output_dir=f"aai_ModernBERT_{task}_ft",
        learning_rate=lr,
        per_device_train_batch_size=train_bsz,
        per_device_eval_batch_size=val_bsz,
        num_train_epochs=n_epochs,
        lr_scheduler_type="linear",
        optim="adamw_torch",
        adam_beta1=betas[0],
        adam_beta2=betas[1],
        adam_epsilon=eps,
        weight_decay=wd,
        logging_strategy="epoch",
        eval_strategy="epoch",
        save_strategy="epoch",
        load_best_model_at_end=True,
        bf16=True,
        bf16_full_eval=True,
        push_to_hub=False,
    )

    trainer = Trainer(
        model=hf_model,
        args=training_args,
        train_dataset=tokenized_datasets[train_ds_name],
        eval_dataset=tokenized_datasets[valid_ds_name],
        processing_class=hf_tokenizer,
        data_collator=hf_data_collator,
        compute_metrics=task_compute_metrics,
    )

    # Add callback to trainer
    metrics_callback = MetricsCallback()
    trainer.add_callback(metrics_callback)

    trainer.train()

    # 7. Get the training results and hyperparameters
    train_history_df = pd.DataFrame(metrics_callback.training_history["train"])
    train_history_df = train_history_df.add_prefix("train_")
    eval_history_df = pd.DataFrame(metrics_callback.training_history["eval"])
    train_res_df = pd.concat([train_history_df, eval_history_df], axis=1)

    args_df = pd.DataFrame([training_args.to_dict()])

    # 8. Cleanup (optional)
    if do_cleanup:
        cleanup(things_to_delete=[trainer, hf_model, hf_tokenizer, tokenized_datasets, raw_datasets])

    return train_res_df, args_df, hf_model, hf_tokenizer

In [ ]:
checkpoint = "output_model/modernbert-mask-1b"  # "answerdotai/ModernBERT-base", "answerdotai/ModernBERT-large"
learning_rates = [1e-5, 5e-5, 8e-5]
weight_decays = [1e-6, 5e-6, 8e-6, 1e-5]

In [13]:
checkpoint = "output_model/modernbert-kan-1.5b"  # "answerdotai/ModernBERT-base", "answerdotai/ModernBERT-large"
learning_rates = [5e-5, 8e-5]
weight_decays = [1e-6, 5e-6, 8e-6, 1e-5]

hf_tokenizer = AutoTokenizer.from_pretrained(checkpoint)
for task in glue_tasks.keys():
    for lr in learning_rates:
        for wd in weight_decays:
            print(f"----- Finetuning {task} | lr={lr} | wd={wd} -----")
            train_res_df, args_df, hf_model, hf_tokenizer = finetune_glue_task(
                lr, wd, task, checkpoint=checkpoint, train_subset=None, do_cleanup=True
            )

            print(":: Results ::")
            display(train_res_df)
            display(args_df)

----- Finetuning qqp | lr=5e-05 | wd=1e-06 -----


Map: 100%|██████████| 390965/390965 [00:17<00:00, 21860.12 examples/s]
Some weights of ModernBertForSequenceClassification were not initialized from the model checkpoint at output_model/modernbert-kan-1.5b and are newly initialized: ['classifier.bias', 'classifier.weight', 'head.dense.weight', 'model.layers.0.attn.Wo.weight', 'model.layers.0.attn.Wqkv.weight', 'model.layers.0.mlp.Wi.weight', 'model.layers.0.mlp.Wo.weight', 'model.layers.1.attn.Wo.weight', 'model.layers.1.attn.Wqkv.weight', 'model.layers.1.mlp.Wi.weight', 'model.layers.1.mlp.Wo.weight', 'model.layers.2.attn.Wo.weight', 'model.layers.2.attn.Wqkv.weight', 'model.layers.2.mlp.Wi.weight', 'model.layers.2.mlp.Wo.weight', 'model.layers.3.attn.Wo.weight', 'model.layers.3.attn.Wqkv.weight', 'model.layers.3.mlp.Wi.weight', 'model.layers.3.mlp.Wo.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.
The tokenizer has new PAD/BOS/EOS tokens that differ from the mode

Epoch,Training Loss,Validation Loss,F1 Score,Accuracy Score
1,0.530700,0.503569,0.632384,0.751571
2,0.481400,0.478566,0.644241,0.765620
3,0.448900,0.456189,0.697899,0.778457
4,0.421300,0.441895,0.705699,0.789760
5,0.401000,0.432526,0.720123,0.793149
6,0.383900,0.426735,0.721795,0.800618
7,0.369000,0.425275,0.730682,0.800000
8,0.356800,0.422153,0.726984,0.804304
9,0.347000,0.421607,0.731350,0.802968
10,0.339600,0.425973,0.733045,0.804897


:: Results ::


,train_loss,train_grad_norm,train_learning_rate,train_epoch,eval_loss,eval_f1_score,eval_accuracy_score
0,0.5307,4.428421,4.500044e-05,1.0,0.503569,0.632384,0.751571
1,0.4814,10.119098,4.000044e-05,2.0,0.478566,0.644241,0.765620
2,0.4489,9.667716,3.500044e-05,3.0,0.456189,0.697899,0.778457
3,0.4213,17.647268,3.000044e-05,4.0,0.441895,0.705699,0.789760
4,0.4010,5.257286,2.500044e-05,5.0,0.432526,0.720123,0.793149
5,0.3839,6.184324,2.000044e-05,6.0,0.426735,0.721795,0.800618
6,0.3690,5.671434,1.500044e-05,7.0,0.425275,0.730682,0.800000
7,0.3568,9.904021,1.000044e-05,8.0,0.422153,0.726984,0.804304
8,0.3470,3.979645,5.000440e-06,9.0,0.421607,0.731350,0.802968
9,0.3396,14.293823,4.397151e-10,10.0,0.425973,0.733045,0.804897


...

Some weights of ModernBertForSequenceClassification were not initialized from the model checkpoint at output_model/modernbert-kan-1.5b and are newly initialized: ['classifier.bias', 'classifier.weight', 'head.dense.weight', 'model.layers.0.attn.Wo.weight', 'model.layers.0.attn.Wqkv.weight', 'model.layers.0.mlp.Wi.weight', 'model.layers.0.mlp.Wo.weight', 'model.layers.1.attn.Wo.weight', 'model.layers.1.attn.Wqkv.weight', 'model.layers.1.mlp.Wi.weight', 'model.layers.1.mlp.Wo.weight', 'model.layers.2.attn.Wo.weight', 'model.layers.2.attn.Wqkv.weight', 'model.layers.2.mlp.Wi.weight', 'model.layers.2.mlp.Wo.weight', 'model.layers.3.attn.Wo.weight', 'model.layers.3.attn.Wqkv.weight', 'model.layers.3.mlp.Wi.weight', 'model.layers.3.mlp.Wo.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.
The tokenizer has new PAD/BOS/EOS tokens that differ from the model config and generation config. The model config and generation config 

Epoch,Training Loss,Validation Loss,Accuracy Score
1,0.749800,0.691836,0.559567
2,0.681700,0.715767,0.527076
3,0.643000,0.693733,0.545126
4,0.592200,0.727088,0.534296
5,0.513200,0.805836,0.519856
6,0.446000,0.892868,0.509025
7,0.374400,1.001510,0.494585
8,0.324200,1.066062,0.519856
9,0.293300,1.070396,0.498195
10,0.264900,1.090363,0.501805


:: Results ::


,train_loss,train_grad_norm,train_learning_rate,train_epoch,eval_loss,eval_accuracy_score
0,0.7498,19.173941,7.210256e-05,1.0,0.691836,0.559567
1,0.6817,9.617178,6.410256e-05,2.0,0.715767,0.527076
2,0.6430,27.365223,5.610256e-05,3.0,0.693733,0.545126
3,0.5922,19.501883,4.810256e-05,4.0,0.727088,0.534296
4,0.5132,14.722438,4.010256e-05,5.0,0.805836,0.519856
5,0.4460,19.065058,3.210256e-05,6.0,0.892868,0.509025
6,0.3744,45.435341,2.410256e-05,7.0,1.001510,0.494585
7,0.3242,42.566277,1.610256e-05,8.0,1.066062,0.519856
8,0.2933,14.105376,8.102564e-06,9.0,1.070396,0.498195
9,0.2649,18.517746,1.025641e-07,10.0,1.090363,0.501805


,output_dir,overwrite_output_dir,do_train,do_eval,do_predict,eval_strategy,prediction_loss_only,per_device_train_batch_size,per_device_eval_batch_size,per_gpu_train_batch_size,...,include_tokens_per_second,include_num_input_tokens_seen,neftune_noise_alpha,optim_target_modules,batch_eval_metrics,eval_on_start,use_liger_kernel,liger_kernel_config,eval_use_gather_object,average_tokens_across_devices
0,aai_ModernBERT_rte_ft,False,False,True,False,epoch,False,32,32,None,...,False,False,None,None,False,False,False,None,False,False


In [22]:
checkpoint = "output_model/modernbert-kan-1.5b"  # "answerdotai/ModernBERT-base", "answerdotai/ModernBERT-large"
learning_rates = [5e-5, 8e-5]
weight_decays = [1e-6, 5e-6, 8e-6, 1e-5]

hf_tokenizer = AutoTokenizer.from_pretrained(checkpoint)
for task in glue_tasks.keys():
    for lr in learning_rates:
        for wd in weight_decays:
            print(f"----- Finetuning {task} | lr={lr} | wd={wd} -----")
            train_res_df, args_df, hf_model, hf_tokenizer = finetune_glue_task(
                lr, wd, task, checkpoint=checkpoint, train_subset=None, do_cleanup=True
            )

            print(":: Results ::")
            display(train_res_df)
            display(args_df)

----- Finetuning stsb | lr=5e-05 | wd=1e-06 -----


Map: 100%|██████████| 1500/1500 [00:00<00:00, 20970.05 examples/s]
Some weights of ModernBertForSequenceClassification were not initialized from the model checkpoint at output_model/modernbert-kan-1.5b and are newly initialized: ['classifier.bias', 'classifier.weight', 'head.dense.weight', 'model.layers.0.attn.Wo.weight', 'model.layers.0.attn.Wqkv.weight', 'model.layers.0.mlp.Wi.weight', 'model.layers.0.mlp.Wo.weight', 'model.layers.1.attn.Wo.weight', 'model.layers.1.attn.Wqkv.weight', 'model.layers.1.mlp.Wi.weight', 'model.layers.1.mlp.Wo.weight', 'model.layers.2.attn.Wo.weight', 'model.layers.2.attn.Wqkv.weight', 'model.layers.2.mlp.Wi.weight', 'model.layers.2.mlp.Wo.weight', 'model.layers.3.attn.Wo.weight', 'model.layers.3.attn.Wqkv.weight', 'model.layers.3.mlp.Wi.weight', 'model.layers.3.mlp.Wo.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.
The tokenizer has new PAD/BOS/EOS tokens that differ from the model co

Epoch,Training Loss,Validation Loss,Pearsonr,Spearmanr,Accuracy Score
1,2.519400,2.665220,0.119624,0.133326,0.571333
2,2.037200,2.620943,0.121742,0.119475,0.561333
3,1.896600,2.611137,0.077178,0.080723,0.552667
4,1.802600,2.562148,0.144315,0.137203,0.578000
5,1.701300,2.536248,0.154153,0.144463,0.580667
6,1.586900,2.630199,0.130979,0.132340,0.581333
7,1.513700,2.650815,0.134476,0.132864,0.578667
8,1.448300,2.705155,0.152244,0.151331,0.580000
9,1.397200,2.611263,0.147366,0.146839,0.594667
10,1.358700,2.650681,0.145670,0.145455,0.588000


:: Results ::


,train_loss,train_grad_norm,train_learning_rate,train_epoch,eval_loss,eval_pearsonr,eval_spearmanr,eval_accuracy_score
0,2.5194,132.235825,4.502778e-05,1.0,2.665220,0.119624,0.133326,0.571333
1,2.0372,110.535271,4.002778e-05,2.0,2.620943,0.121742,0.119475,0.561333
2,1.8966,91.087616,3.502778e-05,3.0,2.611137,0.077178,0.080723,0.552667
3,1.8026,72.223007,3.002778e-05,4.0,2.562148,0.144315,0.137203,0.578000
4,1.7013,55.210609,2.502778e-05,5.0,2.536248,0.154153,0.144463,0.580667
5,1.5869,86.483932,2.002778e-05,6.0,2.630199,0.130979,0.132340,0.581333
6,1.5137,55.078320,1.502778e-05,7.0,2.650815,0.134476,0.132864,0.578667
7,1.4483,68.377365,1.002778e-05,8.0,2.705155,0.152244,0.151331,0.580000
8,1.3972,74.766937,5.027778e-06,9.0,2.611263,0.147366,0.146839,0.594667
9,1.3587,55.797760,2.777778e-08,10.0,2.650681,0.145670,0.145455,0.588000


...

Some weights of ModernBertForSequenceClassification were not initialized from the model checkpoint at output_model/modernbert-kan-1.5b and are newly initialized: ['classifier.bias', 'classifier.weight', 'head.dense.weight', 'model.layers.0.attn.Wo.weight', 'model.layers.0.attn.Wqkv.weight', 'model.layers.0.mlp.Wi.weight', 'model.layers.0.mlp.Wo.weight', 'model.layers.1.attn.Wo.weight', 'model.layers.1.attn.Wqkv.weight', 'model.layers.1.mlp.Wi.weight', 'model.layers.1.mlp.Wo.weight', 'model.layers.2.attn.Wo.weight', 'model.layers.2.attn.Wqkv.weight', 'model.layers.2.mlp.Wi.weight', 'model.layers.2.mlp.Wo.weight', 'model.layers.3.attn.Wo.weight', 'model.layers.3.attn.Wqkv.weight', 'model.layers.3.mlp.Wi.weight', 'model.layers.3.mlp.Wo.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.
The tokenizer has new PAD/BOS/EOS tokens that differ from the model config and generation config. The model config and generation config 

Epoch,Training Loss,Validation Loss,Pearsonr,Spearmanr,Accuracy Score
1,2.444200,2.596641,0.130662,0.134036,0.580000
2,1.967700,2.599096,0.138391,0.129635,0.554000
3,1.805900,2.817280,0.094159,0.101228,0.541333
4,1.674200,2.431752,0.170331,0.163641,0.604000
5,1.531100,2.740855,0.167199,0.162911,0.559333
6,1.373900,2.573315,0.161535,0.166688,0.598667
7,1.264600,2.841943,0.156453,0.158819,0.570000
8,1.181100,2.834309,0.169436,0.176334,0.586000
9,1.116400,2.716790,0.167857,0.173849,0.594000
10,1.073800,2.738267,0.164031,0.171321,0.595333


:: Results ::


,train_loss,train_grad_norm,train_learning_rate,train_epoch,eval_loss,eval_pearsonr,eval_spearmanr,eval_accuracy_score
0,2.4442,106.219749,7.204444e-05,1.0,2.596641,0.130662,0.134036,0.580000
1,1.9677,84.785568,6.404444e-05,2.0,2.599096,0.138391,0.129635,0.554000
2,1.8059,54.432022,5.604444e-05,3.0,2.817280,0.094159,0.101228,0.541333
3,1.6742,35.723022,4.804444e-05,4.0,2.431752,0.170331,0.163641,0.604000
4,1.5311,45.546944,4.004444e-05,5.0,2.740855,0.167199,0.162911,0.559333
5,1.3739,75.478722,3.204444e-05,6.0,2.573315,0.161535,0.166688,0.598667
6,1.2646,43.953743,2.404444e-05,7.0,2.841943,0.156453,0.158819,0.570000
7,1.1811,49.666836,1.604444e-05,8.0,2.834309,0.169436,0.176334,0.586000
8,1.1164,44.424091,8.044444e-06,9.0,2.716790,0.167857,0.173849,0.594000
9,1.0738,42.302830,4.444444e-08,10.0,2.738267,0.164031,0.171321,0.595333


,output_dir,overwrite_output_dir,do_train,do_eval,do_predict,eval_strategy,prediction_loss_only,per_device_train_batch_size,per_device_eval_batch_size,per_gpu_train_batch_size,...,include_tokens_per_second,include_num_input_tokens_seen,neftune_noise_alpha,optim_target_modules,batch_eval_metrics,eval_on_start,use_liger_kernel,liger_kernel_config,eval_use_gather_object,average_tokens_across_devices
0,aai_ModernBERT_stsb_ft,False,False,True,False,epoch,False,32,32,None,...,False,False,None,None,False,False,False,None,False,False


# K-Folds

In [30]:
from sklearn.model_selection import StratifiedKFold, KFold
from transformers import set_seed
import tempfile
from datasets import concatenate_datasets
import os

def finetune_glue_task_kfold(
    lr, wd, task: str, 
    checkpoint: str = "answerdotai/ModernBERT-base", 
    train_subset: int | None = None, 
    do_cleanup: bool = True,
    k_folds: int = 5,
    random_seed: int = 42,
    train_bsz: int = 32,  # ✅ Add as parameter
    val_bsz: int = 32,    # ✅ Add as parameter  
    n_epochs: int = 2     # ✅ Add as parameter
):
    """Finetune GLUE task with k-fold cross-validation"""
    set_seed(random_seed)
    
    # ✅ Check if checkpoint path exists
    if not os.path.exists(checkpoint):
        print(f"Error: Checkpoint path '{checkpoint}' does not exist!")
        print("Please check the path or use a valid HuggingFace model identifier.")
        return None, None, None, None
    
    # 1. Load the task metadata
    task_meta = glue_tasks[task]
    train_ds_name = task_meta["dataset_names"]["train"]
    valid_ds_name = task_meta["dataset_names"]["valid"]
    task_inputs = task_meta["inputs"]
    n_labels = task_meta["n_labels"]
    task_metrics = task_meta["metric_funcs"]

    # 2. Load and combine datasets
    raw_datasets = load_dataset("glue", task.split("-")[0] if "-" in task else task)
    
    # ✅ Combine train + validation datasets
    combined_dataset = concatenate_datasets([
        raw_datasets[train_ds_name], 
        raw_datasets[valid_ds_name]
    ])
    
    print(f"Combined dataset size: {len(combined_dataset)}")
    print(f"Original train: {len(raw_datasets[train_ds_name])}, val: {len(raw_datasets[valid_ds_name])}")
    
    if train_subset is not None and len(combined_dataset) > train_subset:
        combined_dataset = combined_dataset.shuffle(seed=random_seed).select(range(train_subset))

    # Get label maps from original training set
    id2label, label2id = get_label_maps(raw_datasets, train_ds_name)
    
    # ✅ Create k-fold splits WITHOUT pandas - use dataset.select() directly
    if task == "stsb":
        # Regression - use regular KFold
        kfold = KFold(n_splits=k_folds, shuffle=True, random_state=random_seed)
        indices = list(range(len(combined_dataset)))
        splits = list(kfold.split(indices))
    else:
        # Classification - use StratifiedKFold
        labels = combined_dataset['label']
        kfold = StratifiedKFold(n_splits=k_folds, shuffle=True, random_state=random_seed)
        splits = list(kfold.split(range(len(combined_dataset)), labels))
    
    fold_results = []
    all_train_dfs = []
    all_args_dfs = []
    
    print(f"Starting {k_folds}-fold CV for {task}")
    
    # Clear original dataset references
    del raw_datasets
    gc.collect()
    
    # Process each fold
    for fold, (train_idx, val_idx) in enumerate(splits):
        print(f"\n--- Fold {fold + 1}/{k_folds} ---")
        print(f"Train size: {len(train_idx)}, Val size: {len(val_idx)}")
        
        try:
            # ✅ Use efficient dataset.select() - no pandas conversion needed!
            fold_train_dataset = combined_dataset.select(train_idx)
            fold_val_dataset = combined_dataset.select(val_idx)
            
            # Run single fold training
            train_res_df, args_df, model, tokenizer = _single_fold_training(
                lr, wd, task, checkpoint, fold_train_dataset, fold_val_dataset,
                task_inputs, n_labels, task_metrics, id2label, label2id, 
                do_cleanup, random_seed, fold + 1, train_bsz, val_bsz, n_epochs  # ✅ Pass parameters
            )
            
            # Store results
            train_res_df['fold'] = fold + 1
            args_df['fold'] = fold + 1
            
            all_train_dfs.append(train_res_df)
            all_args_dfs.append(args_df)
            
            # Extract final metrics for summary
            if len(train_res_df) > 0:
                final_metrics = train_res_df.iloc[-1].to_dict()
                final_metrics['fold'] = fold + 1
                fold_results.append(final_metrics)
            
            # Cleanup fold-specific data
            del fold_train_dataset, fold_val_dataset
            del model, tokenizer
            gc.collect()
            torch.cuda.empty_cache()
            
        except Exception as e:
            print(f"Error in fold {fold + 1}: {e}")
            import traceback
            traceback.print_exc()
    
    # Cleanup combined dataset
    del combined_dataset
    gc.collect()
    
    # Aggregate results
    if fold_results:
        results_df = pd.DataFrame(fold_results)
        combined_train_df = pd.concat(all_train_dfs, ignore_index=True)
        combined_args_df = pd.concat(all_args_dfs, ignore_index=True)
        
        # Calculate summary statistics
        summary_stats = {}
        numeric_cols = results_df.select_dtypes(include=[np.number]).columns
        for col in numeric_cols:
            if col != 'fold':
                summary_stats[f'{col}_mean'] = results_df[col].mean()
                summary_stats[f'{col}_std'] = results_df[col].std()
        
        summary_df = pd.DataFrame([summary_stats])
        
        print(f"\n=== {task} K-Fold Results Summary ===")
        print("Individual Fold Results:")
        display(results_df)
        print("\nMean ± Std:")
        display(summary_df)
        
        return combined_train_df, combined_args_df, results_df, summary_df
    else:
        print("No successful folds completed!")
        return None, None, None, None


def _single_fold_training(
    lr, wd, task, checkpoint, train_dataset, val_dataset,
    task_inputs, n_labels, task_metrics, id2label, label2id, 
    do_cleanup, random_seed, fold_num, train_bsz, val_bsz, n_epochs  # ✅ Add missing parameters
):
    """Helper function for single fold training"""
    
    # ✅ Check checkpoint path again before loading
    if not os.path.exists(checkpoint):
        raise ValueError(f"Checkpoint path '{checkpoint}' does not exist!")
    
    # Load tokenizer
    hf_tokenizer = AutoTokenizer.from_pretrained(checkpoint)
    
    # Create fresh preprocessing function
    def fold_preprocess_function(examples):
        inps = [examples[inp] for inp in task_inputs]
        tokenized = hf_tokenizer(*inps, truncation=True)
        return tokenized
    
    # Tokenize datasets
    fold_train_tokenized = train_dataset.map(
        fold_preprocess_function, 
        batched=True,
        remove_columns=[col for col in train_dataset.column_names if col != 'label'],
        desc="Tokenizing train"
    )
    fold_val_tokenized = val_dataset.map(
        fold_preprocess_function, 
        batched=True,
        remove_columns=[col for col in val_dataset.column_names if col != 'label'],
        desc="Tokenizing val"
    )

    # Load model
    model_additional_kwargs = {"id2label": id2label, "label2id": label2id} if id2label and label2id else {}
    hf_model = AutoModelForSequenceClassification.from_pretrained(
        checkpoint, 
        num_labels=n_labels, 
        **model_additional_kwargs,
        dtype=torch.bfloat16,  # ✅ Use 'dtype' instead of 'torch_dtype'
        local_files_only=True  # ✅ Force local loading
    )

    hf_data_collator = DataCollatorWithPadding(tokenizer=hf_tokenizer)

    # ✅ Define betas and eps locally
    betas = (0.9, 0.98)
    eps = 1e-6

    # Training arguments
    with tempfile.TemporaryDirectory() as temp_dir:
        training_args = TrainingArguments(
            output_dir=temp_dir,
            learning_rate=lr,
            per_device_train_batch_size=train_bsz,  # ✅ Now available as parameter
            per_device_eval_batch_size=val_bsz,     # ✅ Now available as parameter
            num_train_epochs=n_epochs,              # ✅ Now available as parameter
            lr_scheduler_type="linear",
            optim="adamw_torch",
            adam_beta1=betas[0],
            adam_beta2=betas[1],
            adam_epsilon=eps,
            weight_decay=wd,
            logging_strategy="no",
            eval_strategy="epoch",
            save_strategy="no",
            load_best_model_at_end=False,
            bf16=True,
            bf16_full_eval=True,
            push_to_hub=False,
            report_to=None,
            seed=random_seed,
            dataloader_pin_memory=False,
            dataloader_num_workers=0,
        )

        trainer = Trainer(
            model=hf_model,
            args=training_args,
            train_dataset=fold_train_tokenized,
            eval_dataset=fold_val_tokenized,
            processing_class=hf_tokenizer,
            data_collator=hf_data_collator,
            compute_metrics=partial(compute_metrics, task_metrics=task_metrics),
        )

        # Add callback
        metrics_callback = MetricsCallback()
        trainer.add_callback(metrics_callback)

        trainer.train()

        # Get results
        train_history_df = pd.DataFrame(metrics_callback.training_history["train"])
        train_history_df = train_history_df.add_prefix("train_")
        eval_history_df = pd.DataFrame(metrics_callback.training_history["eval"])
        train_res_df = pd.concat([train_history_df, eval_history_df], axis=1)

        args_df = pd.DataFrame([training_args.to_dict()])

        # Cleanup
        if do_cleanup:
            if hasattr(hf_model, 'cpu'):
                hf_model.cpu()
            cleanup_items = [trainer, hf_tokenizer, fold_train_tokenized, fold_val_tokenized]
            for item in cleanup_items:
                try:
                    del item
                except:
                    pass
            gc.collect()
            torch.cuda.empty_cache()

        return train_res_df, args_df, hf_model, hf_tokenizer


# ✅ Run with correct parameter passing
checkpoint = "output_model/modernbert-diffusion-1.5b"

# Check if the path exists
if os.path.exists(checkpoint):
    print(f"✅ Checkpoint found at: {checkpoint}")
    
    # Run k-fold cross-validation with explicit parameters
    for task in glue_tasks.keys():
        lr = glue_tasks[task]['learning_rate']
        wd = glue_tasks[task]['weight_decay']
        epochs = glue_tasks[task]['epochs']
        print(f"----- Finetuning {task} | lr={lr} | wd={wd} | epochs={epochs} -----")
        train_res_df, args_df, fold_results, summary_stats = finetune_glue_task_kfold(
            lr=lr,
            wd=wd,
            task=task,
            checkpoint=checkpoint,
            k_folds=2,
            train_subset=None,  # Use all data
            train_bsz=32,       # ✅ Explicit batch size
            val_bsz=32,         # ✅ Explicit batch size
            n_epochs=epochs
        )
else:
    print(f"❌ Checkpoint NOT found at: {checkpoint}")
    print("Available options:")
    print("1. Use a HuggingFace model: checkpoint = 'answerdotai/ModernBERT-base'")
    print("2. Fix the local path")
    print("3. Check if the model was saved correctly")
    
    # Let's check if there are any models in the output_model directory
    output_dir = "output_model"
    if os.path.exists(output_dir):
        print(f"\nContents of {output_dir}:")
        for item in os.listdir(output_dir):
            item_path = os.path.join(output_dir, item)
            if os.path.isdir(item_path):
                print(f"  📁 {item}/")
                # Check if it contains model files
                sub_files = os.listdir(item_path)
                if any(f.endswith(('.bin', '.safetensors')) for f in sub_files):
                    print(f"    ✅ Contains model files: {[f for f in sub_files if f.endswith(('.bin', '.safetensors', '.json'))]}")
            else:
                print(f"  📄 {item}")

✅ Checkpoint found at: output_model/modernbert-diffusion-1.5b
----- Finetuning cola | lr=5e-05 | wd=8e-06 | epochs=2 -----
Combined dataset size: 9594
Original train: 8551, val: 1043
Starting 2-fold CV for cola


Some weights of ModernBertForSequenceClassification were not initialized from the model checkpoint at output_model/modernbert-diffusion-1.5b and are newly initialized: ['classifier.bias', 'classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.
The tokenizer has new PAD/BOS/EOS tokens that differ from the model config and generation config. The model config and generation config were aligned accordingly, being updated with the tokenizer's values. Updated tokens: {'eos_token_id': None, 'bos_token_id': None}.



--- Fold 1/2 ---
Train size: 4797, Val size: 4797


Epoch,Training Loss,Validation Loss,Matthews Corrcoef,Accuracy Score
1,No log,1.091711,0.020737,0.608297
2,No log,1.071679,0.024101,0.607255


Some weights of ModernBertForSequenceClassification were not initialized from the model checkpoint at output_model/modernbert-diffusion-1.5b and are newly initialized: ['classifier.bias', 'classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.
The tokenizer has new PAD/BOS/EOS tokens that differ from the model config and generation config. The model config and generation config were aligned accordingly, being updated with the tokenizer's values. Updated tokens: {'eos_token_id': None, 'bos_token_id': None}.


...

----- Finetuning stsb | lr=8e-05 | wd=1e-06 | epochs=7 -----
Combined dataset size: 7249
Original train: 5749, val: 1500
Starting 2-fold CV for stsb


Some weights of ModernBertForSequenceClassification were not initialized from the model checkpoint at output_model/modernbert-diffusion-1.5b and are newly initialized: ['classifier.bias', 'classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.
The tokenizer has new PAD/BOS/EOS tokens that differ from the model config and generation config. The model config and generation config were aligned accordingly, being updated with the tokenizer's values. Updated tokens: {'eos_token_id': None, 'bos_token_id': None}.



--- Fold 1/2 ---
Train size: 3624, Val size: 3625


Epoch,Training Loss,Validation Loss,Pearsonr,Spearmanr,Accuracy Score
1,No log,3.244274,0.144766,0.145722,0.554759
2,No log,2.753454,0.191066,0.192461,0.575448


KeyboardInterrupt: 

# KAN Classification

In [ ]:
from transformers import PreTrainedModel, AutoModel, AutoModelForSequenceClassification
from transformers.models.auto import AutoConfig
import torch
import torch.nn as nn


# Add this before your model loading
def load_kan_model_for_classification(checkpoint, num_labels, **kwargs):
    try:
        # Try standard loading first
        return AutoModelForSequenceClassification.from_pretrained(
            checkpoint, num_labels=num_labels, **kwargs
        )
    except Exception as e:
        print(f"Standard loading failed: {e}")
        print("Attempting to load as custom KAN model...")
        
        # Load base model and add classification head
        base_model = AutoModel.from_pretrained(checkpoint)
        
        # Create a simple wrapper
        class KANSequenceClassifier(torch.nn.Module):
            def __init__(self, base_model, num_labels):
                super().__init__()
                self.base_model = base_model
                self.classifier = torch.nn.Linear(base_model.config.hidden_size, num_labels)
                self.config = base_model.config
                self.config.num_labels = num_labels
                if 'id2label' in kwargs: self.config.id2label = kwargs['id2label']
                if 'label2id' in kwargs: self.config.label2id = kwargs['label2id']
            
            def forward(self, **inputs):
                outputs = self.base_model(**inputs)
                pooled_output = outputs.pooler_output
                logits = self.classifier(pooled_output)
                return type('obj', (object,), {'logits': logits, 'loss': None})()
        
        return KANSequenceClassifier(base_model, num_labels)

# Then replace your model loading with:
hf_model = load_kan_model_for_classification(
    checkpoint, num_labels=n_labels, **model_additional_kwargs
)